In [8]:
import torch
import random

import pandas as pd
import numpy as np

from etna.datasets.tsdataset import TSDataset
from etna.pipeline import AutoRegressivePipeline
from etna.transforms import LagTransform

import warnings


def set_seed(seed: int = 42):
    """Set random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


warnings.filterwarnings("ignore")

In [9]:
from config.experiment_config import ExperimentConfig
from config.model_config import ModelConfig

model_config = ModelConfig()
experiment_config = ExperimentConfig()

In [10]:
data = pd.read_csv(experiment_config.PATH_DATA / experiment_config.DATASET.value)
data["datetime"] = pd.to_datetime(data["datetime"])
data = data.sort_values(by="datetime")
data = data.set_index("datetime")
data.index = data.index.tz_localize(None)

data["segment"] = "segment_A"

In [11]:
returns = data.loc[:, experiment_config.RETURN_COLUMN].iloc[1:]

vols = data.loc[:, experiment_config.VOL_COLUMN].iloc[1:]

feature_columns = data.columns.difference(
    [experiment_config.RETURN_COLUMN, experiment_config.VOL_COLUMN]
).tolist()
features = data[feature_columns].shift(1).iloc[1:]

if model_config.n_features is None:
    model_config.n_features = len(feature_columns)

if model_config.n_unique_features is None:
    unique_columns = ["_".join(column.split("_")[:-1]) for column in feature_columns]
    model_config.n_unique_features = np.unique(unique_columns).shape[0]

data = data.reset_index()
data = data.rename(columns={"datetime": "timestamp"})
data = data.set_index("timestamp")

In [12]:
from etna.models.nn import PatchTSModel


def train_n_beats(train_df: pd.DataFrame, test_df: pd.DataFrame):
    set_seed()

    df = TSDataset.to_dataset(train_df)
    ts = TSDataset(df, freq="h")

    num_lags = 1
    transform_lag = LagTransform(
        in_column="target",
        lags=[1 + i for i in range(num_lags)],
        out_column="target_lag",
    )

    model_deepar = PatchTSModel(
        decoder_length=12,
        encoder_length=10,
        patch_len=1,
        trainer_params=dict(max_epochs=10),
        lr=1e-3,
    )

    pipeline_deepar = AutoRegressivePipeline(
        model=model_deepar,
        horizon=test_df.shape[0],
        transforms=[transform_lag],
    )

    _ = pipeline_deepar.fit(ts=ts)
    forecast = pipeline_deepar.forecast()
    pred_vol = forecast.loc[:, ("segment_A", "target")].to_numpy()

    return pred_vol

In [13]:
def run_backtest():
    step = experiment_config.ROLLING_STEP_DAYS

    train_start = experiment_config.TRAIN_START_DATE
    train_end = experiment_config.FIRST_TRAIN_END_DATE

    rolling_results = []
    while train_end <= data.index[-1]:
        last_train_end = train_end
        train_data = (
            data.resample("h").ffill().loc[train_start:train_end, :].reset_index()
        )
        train_data = train_data.rename(columns={"vol": "target"})
        train_start = (
            train_start + pd.Timedelta(days=step)
            if experiment_config.EXPANDING
            else train_start
        )
        train_end = train_end + pd.Timedelta(days=step)

        test_data = data.loc[
            last_train_end + pd.Timedelta(milliseconds=1) : train_end, :
        ].reset_index()

        if len(train_data) > 0 and len(test_data) > 0:
            forecast = train_n_beats(
                train_df=train_data,
                test_df=test_data,
            )

            rolling_results.append(
                [last_train_end, test_data["vol"].to_numpy(), forecast]
            )

    return pd.DataFrame(
        rolling_results, columns=["datetime", "true_vols", "model_preds"]
    ).set_index("datetime")

In [14]:
results = run_backtest()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type       | Params
------------------------------------------
0 | loss       | MSELoss    | 0     
1 | model      | Sequential | 397 K 
2 | projection | Sequential | 1.3 K 
------------------------------------------
398 K     Trainable params
0         Non-trainable params
398 K     Total params
1.596     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 5/5 [00:02<00:00,  1.67it/s, loss=0.0298]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 5/5 [00:03<00:00,  1.66it/s, loss=0.0298]


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type       | Params
------------------------------------------
0 | loss       | MSELoss    | 0     
1 | model      | Sequential | 397 K 
2 | projection | Sequential | 1.3 K 
------------------------------------------
398 K     Trainable params
0         Non-trainable params
398 K     Total params
1.596     Total estimated model params size (MB)


...

Epoch 9: 100%|██████████| 5/5 [00:02<00:00,  1.81it/s, loss=0.0292]


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type       | Params
------------------------------------------
0 | loss       | MSELoss    | 0     
1 | model      | Sequential | 397 K 
2 | projection | Sequential | 1.3 K 
------------------------------------------
398 K     Trainable params
0         Non-trainable params
398 K     Total params
1.596     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 5/5 [00:02<00:00,  1.88it/s, loss=0.0292]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 5/5 [00:02<00:00,  1.88it/s, loss=0.0292]


In [15]:
results.to_csv("PatchTS_Features.csv")